![image](https://user-images.githubusercontent.com/1651790/221876073-61ef4edb-adcd-4f10-b3fc-8ddc24918ea1.png)

In [ ]:
# install ng_ai in the first run
%pip install ng_ai[networkx] matplotlib

## AI Suite NetworkX Engine Examples
### read data with NetowrkX engine, query mode

In this example, we are leveraging the NetworkX Engine of NebulaGraph AI Suite, with the GraphD Query mode.

#### Step 1, get graph in Query Mode

We will scan all edge in type `follow` and `serve` first with props `degree` in `follow` and no props in `serve` as graph: `g`

In [ ]:
from ng_ai import NebulaReader
from ng_ai.config import NebulaGraphConfig

# read data with nebula/networkx engine, query mode
config_dict = {
    "graphd_hosts": "graphd:9669",
    "user": "root",
    "password": "nebula",
    "space": "basketballplayer",
}
config = NebulaGraphConfig(**config_dict)
reader = NebulaReader(engine="nebula", config=config)
reader.query(edges=["follow", "serve"], props=[["degree"], []])
g = reader.read()
g.show(10)
g.draw()

#### Step 2, run Pagerank Algorithm

In [ ]:
pr_result = g.algo.pagerank(reset_prob=0.15, max_iter=10)

#### Step 3, check results of the algorithm


In [ ]:
pr_result

### Write back algo result to NebulaGraph

Assume that we have a result `graph_result` computed with `g.algo.pagerank()`:

```python
{'player102': 0.014770646980811417,
 'player100': 0.02878478843123552,
 'player101': 0.020163880830622937,
 'player129': 0.012381302535422786,
 'player116': 0.015041184157101154,
 'player121': 0.012178909379871223,
 'player128': 0.010197889677928056,
...
}
```

Let's write them back to tag: pagerank(pagerank). So we create a TAG `pagerank` in NebulaGraph on same space with the following schema:

```ngql
CREATE TAG IF NOT EXISTS pagerank (
    pagerank double NOT NULL
);
```

Then, we could write the pagerank result to NebulaGraph, to tag `pagerank` with property `pagerank`:

```python
properties = ["pagerank"]
```
And pass it to NebulaWriter in `nebula` engine and `nebulagraph_vertex` sink

In [ ]:
# Run pagerank Algorithm
graph_result = g.algo.pagerank()

In [9]:
from ng_ai import NebulaWriter

writer = NebulaWriter(
    data=graph_result,
    sink="nebulagraph_vertex",
    config=config,
    engine="nebula",
)

# properties to write
properties = ["pagerank"]

writer.set_options(
    tag="pagerank",
    properties=properties,
    batch_size=256,
    write_mode="insert",
)
# write back to NebulaGraph
writer.write()

Then we could query the result in NebulaGraph:

```cypher
MATCH (v)
RETURN id(v), properties(v).name, v.pagerank.pagerank LIMIT 10;
```

## How to run other algorithm examples

In [ ]:
# get all algorithms
g.algo.get_all_algo()

# get help of each algo
help(g.algo.node2vec)

# call the algo
g.algo.node2vec()